Below is the code to make a library of charge signals throughout the detector using icpc_siggen source code for symmetric HPGe detectors. 

This code will create the library in a pkl file then proceed to open and plot specific waveforms. The signals can be adapted in the config file to include different variations of your choosing. 

In [ ]:
#import subprocess which is the package created for the fieldgen and siggen source code
#below is an example of running one of the config files

import subprocess
subprocess.run("cat config_files/50372.config", shell=True).stdout

In [ ]:
# To get a list of the stester commands, type "help"
f = open("my_stester_commands.txt", "w")
f.write("help\n")
f.write("quit\n")
f.close()
subprocess.run("./stester config_files/50372.config < my_stester_commands.txt", 
               check=False, shell=True).stdout

In [ ]:
import numpy as np
import pandas as pd   
import os  

#Define an array of positions to ititerate through
r= np.arange(0, 46.5, 0.5)
z= np.arange(0, 111.5, 0.5)
phi = round(np.pi/2,3)

#Create an empty dataframe to store the final positions and waveforms           
results_df = pd.DataFrame(columns=['r', 'phi', 'z', 'data_column'])


#Ititerate through all possible r and and z values while keeping phi constant
for i in r:
     for j in z:
            #For each position I create a file with the position name and run it through using stester and subprocess

            #You must create a folder called Normal_Library in your directory for this to work
            f = open("my_stester_commands.txt", "w")
            f.write(f"sig {i} {phi} {j} Normal_Library/normal{i,phi,j}.spe\n")
            f.write("quit\n")
            
            positions = np.vstack((i, phi, j)).T
            print(positions)
            
            f.close()
            #Run all the simulations for each position
            subprocess.run("./stester config_files/50372.config < my_stester_commands.txt", check=False, shell=True).stdout


            filename = f"Normal_Library/normal{i, phi, j}.spe"

            # Check if the file exists before trying to open it
            if os.path.exists(filename):
                with open(filename, 'rb') as f:
                    yraw = f.read(400 * 4)
                    y = np.array(struct.unpack('f' * 400, yraw))
                #Save positions and results as row in the dataframe
                new_row = pd.DataFrame([[positions[0][0], positions[0][1], positions[0][2], y.tolist()]],
                                   columns=['r', 'phi', 'z', 'data_column'])
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            else:
                print(f"File not found: {filename}")

#print(results_df)


            

In [ ]:
#Saving the results to a pickle file and opening the file to display the table
import pandas as pd
results_df.to_pickle('output_dataframe.pkl')

df = pd.read_pickle('output_dataframe.pkl')
display(df)

In [ ]:
import itertools
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from ipywidgets import interact, widgets
from matplotlib import colormaps
from matplotlib.cm import ScalarMappable


df = pd.read_pickle('output_dataframe.pkl')#.head(125)
df['r'] = df['r'].round(2)
df['z'] = df['z'].round(2)
#display(df)
df_rounded = df[['r', 'z', 'data_column']].copy()
display(df_rounded)

#The function below is designed to plot the elctric field lines as a 2D color map using the detectors geometry which I have adapted to also
#plot an x for the position of the waveform also being plotted
def plot_position(r_values, z_values):
    data = np.loadtxt("fields/ev_50372.dat")
    # figure out the size of grid the X-Y points
    x = set(data[:,0])  # sets only have one copy of anything, so if there are repeats, they are removed
    y = set(data[:,1])
    z = data[:,3]       # column 3 = field, 2 = potential
    # reshape the zvals array into the appropriate shape, and find the boundaries
    zvals = z.reshape(len(x), len(y))
    zvals[zvals < 0] = 0
    # imshow plots columns and rows opposite to how you'd expect; so transpose them
    zvals = zvals.T
    # stack so we can plot the data from one half of the detector (positive r-values only)
    zvals_neg = np.fliplr(zvals)
    zvals_full = np.hstack((zvals_neg,zvals))
    bounds = (-1 * max(x), max(x), min(y), max(y))

    fig = plt.figure(figsize=(10,7))
    plt.figure(0)
    plt.imshow(zvals_full, norm=colors.LogNorm(),
               extent=bounds,   # set the boundaries of the edges of the 'image' data
               origin="lower",  # tell matplotlib where [0,0] is in the bottom
               cmap='viridis')      # use the 'jet color map scheme
    
    if r_values == 9.29:
        x = 'red'
        
    else:
        x = 'blue'
        
    for z_value in z_values:
        plt.figure(0)
        plt.scatter(r_values, z_value, color=x, marker='x', label=f'Current Position: r={r_values}, z={z_value}')
    
    plt.xlabel("Radial position [mm]", size=13)
    plt.ylabel("Axial position [mm]", labelpad=8,  size=13)
    plt.colorbar()
    #plt.legend()
    
    


# Example usage
r_values1 = 9.29 
z_values = [0.00, 1.12, 2.24, 3.36, 4.48, 5.61, 6.73, 7.85, 8.97, 10.09, 11.21, 12.33, 13.45, 14.57, 15.70, 16.82, 17.94, 19.06, 20.18, 21.30, 22.42, 23.55, 24.67, 25.79, 26.91, 28.03, 29.15, 30.27, 31.39, 32.52, 33.64, 34.76] 
r_values2 = 20.44


In [ ]:
#This function will plot the charge signal for any array of r and z values

z_value = [0.00, 1.12, 2.24, 3.36, 4.48, 5.61, 6.73, 7.85, 8.97, 10.09, 11.21, 12.33, 13.45, 14.57, 15.70, 16.82, 17.94, 19.06, 20.18, 21.30, 22.42, 23.55, 24.67, 25.79, 26.91, 28.03, 29.15, 30.27, 31.39, 32.52, 33.64, 34.76] 


def plotwaveform(r_value1, r_value2, z_values, phi_value=np.pi/2):
    
    plt.figure(figsize=(13, 7))
    
    # Define color maps
    cmap1 = plt.get_cmap('Reds')
    cmap2 = plt.get_cmap('Blues')
    
    # Normalize color maps based on the range of z_values
    norm = colors.Normalize(vmin=min(z_values), vmax=max(z_values))
    
    # Create ScalarMappable objects for colorbars
    sm1 = plt.cm.ScalarMappable(cmap=cmap1, norm=norm)
    sm1.set_array([])
    sm2 = plt.cm.ScalarMappable(cmap=cmap2, norm=norm)
    sm2.set_array([])
    
    # Loop through each z_value
    for z_val in z_values:
        # Filter DataFrame by z_value and r_value1
        filtered_df = df_rounded[(df_rounded['z'] == z_val) & (df_rounded['r'] == r_value1)]
        # Plot waveform for each row in filtered DataFrame
        for _, row in filtered_df.iterrows():
            y = row['data_column']
            x = np.arange(len(y))
            # Choose color from cmap1 if r_value1 is 9.29, otherwise choose from cmap2
            color = cmap1(norm(z_val)) if r_value1 == 9.29 else cmap2(norm(z_val))
            plt.plot(x, y, color=color, label=f'r={r_value1}, phi={phi_value}, z={z_val}')
    
    # Repeat the same process for r_value2
    for z_val in z_values:
        filtered_df = df_rounded[(df_rounded['z'] == z_val) & (df_rounded['r'] == r_value2)]
        for _, row in filtered_df.iterrows():
            y = row['data_column']
            x = np.arange(len(y))
            color = cmap1(norm(z_val)) if r_value2 == 9.29 else cmap2(norm(z_val))
            plt.plot(x, y, color=color, label=f'r={r_value2}, phi={phi_value}, z={z_val}')
    
    plt.xlabel('Time [ns]', size=13)
    plt.ylabel('Charge signal', labelpad=8, size=13)
    plt.title('Siggen Signals\n', fontsize=16)
    
    # Add colorbars to the plot
    cbar1 = plt.colorbar(sm1, ax=plt.gca(), pad=0.01)
    cbar1.set_label('Z Values for r=9.29', size=12)
    cbar2 = plt.colorbar(sm2, ax=plt.gca(), pad=0.15)
    cbar2.set_label('Z Values for r=20.44', size=12)
    
    plt.grid(True)
    # plt.legend()

   
plot_position(r_values1, z_values)
plot_position(r_values2, z_values)

plotwaveform(r_value1=9.29,r_value2=20.44, z_value=z_value)
